# Simultaneous Kelly Criterion

This is a Python implementation of a simultaneous Kelly Criterion optimizer.
This code is based heavily on code available in [this article](https://vegapit.com/article/numerically_solve_kelly_criterion_multiple_simultaneous_bets).

The general expression for the Kelly Criterion of $n$ simultaneous bets is as follows:

- $p_i$ is the probability of winning the $i$th bet
- $f_i$ is the wager amount of the $i$th bet
- $b_i$ is the decimal odds of the $i$th bet, note here that I use the odds as they would be listed at a sportsbook and not net odds as typically used

(Todo)

$\sum_{i=1}^n$

In [1]:
import itertools
import math

import numpy as np
import pandas as pd
import torch
from tqdm import tqdm, trange

In [2]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = "cpu"
device

'cpu'

In [3]:
in_df = pd.read_excel("bet_optimizer_input.xlsx")
num_bets = in_df.shape[0]

In [4]:
bankroll = 9644.82
max_bet = 1.0 * bankroll

# Inputs of weights to tensorflow
probs = torch.tensor(in_df["FiveThirtyEight Probability"]).to(device)
odds = torch.tensor(in_df["Odds"]).to(device)

# Amounts we want to optimize
# wager_amts = torch.tensor(torch.FloatTensor(num_bets).uniform_(0, 1 / num_bets), requires_grad=True)
wager_amts = torch.zeros(num_bets, requires_grad=True, dtype=torch.float64, device=device)

In [5]:
def kc_vec(amts):
    """
    Vectorized compuation of Kelly Criterion for all bets
    """
    amts = amts.clamp(0, max_bet)
    # Generate all possible permutations of winning/losing each bet
    win_permutations = torch.cartesian_prod(*[torch.tensor([0, 1])]*num_bets).to(device)
    
    # Compute probability of winning each bet
    prob_coeff = torch.where(win_permutations == 0, 1 - probs, probs).prod(axis=1)
    # Compute profit of each bet
    wealth = torch.where(win_permutations == 0, -1 * amts, amts * (odds - 1)).sum(axis=1)
    
    return -1 * (prob_coeff * torch.log(bankroll + wealth)).sum()

In [6]:
def loss_closure():
    loss_val = kelly_criterion(wager_amts)
    loss_val.backward()
    return loss_val

In [7]:
epochs = 10000

optimizer = torch.optim.SGD([wager_amts], lr=bankroll / epochs, momentum=0)
optimizer.zero_grad()

min_loss = 0
rec_amts = None

pbar = trange(epochs, miniters=250, unit=" epochs")

for _ in pbar:
    loss = kc_vec(wager_amts)
    
    with torch.no_grad():
        if loss.item() < min_loss:
            min_loss = loss.item()
            rec_amts = wager_amts.detach()
    loss.backward()
    optimizer.step()
    wager_amts.data.clamp_(0, max_bet)
    pbar.set_postfix(refresh=False, loss=loss.data, amounts=wager_amts.data)

100%|█| 10000/10000 [00:05<00:00, 1846.48 epochs/s, amounts=tensor([ 542.6691,   46.7306,  138.9998,   21.5586,   23.013


In [8]:
baseline = math.log(bankroll)
print(f"Log EV of recommended bets: {-min_loss:.4f}")
print(f"Baseline of no bets: {baseline:.4f}")
print(f"Differential: {-min_loss - (baseline):+.4f}")

Log EV of recommended bets: 9.2031
Baseline of no bets: 9.1742
Differential: +0.0289


In [9]:
in_df["Wager Amount"] = rec_amts.cpu()
in_df["Single KC Amount"] = (in_df["FiveThirtyEight Probability"] - (1 - in_df["FiveThirtyEight Probability"]) / (in_df["Odds"] - 1)) * bankroll
in_df["Potential Profit"] = in_df["Wager Amount"] * (in_df["Odds"] - 1)
in_df["Potential Profit %"] = in_df["Potential Profit"] / bankroll

In [10]:
print(f"Max bet: {max_bet:.2f}")

Max bet: 9644.82


In [11]:
in_df.style.format({
    "Odds": '{:f}',
    "FiveThirtyEight Probability": '{:.2%}',
    "Wager Amount": '${:.2f}',
    "Single KC Amount": '${:.2f}',
    "Potential Profit": '${:.2f}',
    "Potential Profit %": '{:.2%}'
})

,Team,Odds,FiveThirtyEight Probability,Wager Amount,Single KC Amount,Potential Profit,Potential Profit %
0,Italy,1.690000,66.00%,$542.67,$1613.06,$374.44,3.88%
1,Germany,1.278000,79.00%,$46.73,$333.75,$12.99,0.13%
2,England,1.182000,87.00%,$139.00,$1501.84,$25.30,0.26%
3,France,1.435000,70.00%,$21.56,$99.77,$9.38,0.10%
4,Portugal,1.182000,85.00%,$23.01,$249.07,$4.19,0.04%
5,Turkey,2.830000,46.00%,$1268.46,$1590.60,$2321.29,24.07%


In [12]:
in_df[["Wager Amount","Single KC Amount"]].sum()

Wager Amount        2041.435979
Single KC Amount    5388.098130
dtype: float64